# Data Preparation

## Data extraction

In this step, raw data was extracted from the BigQuery public dataset.
10,000 rows from one day of data was extracted as a sample, to test out our model.

```
SELECT fullVisitorId, visitNumber, h.eCommerceAction.action_type, prod.productSKU, h.time, h.hitNumber

FROM bigquery-public-data.google_analytics_sample.ga_sessions_20170801, UNNEST(hits) as h, UNNEST(h.product) as prod

Where h.eCommerceAction.action_type != '0'

order by fullVisitorId asc, visitNumber asc, h.time asc

LIMIT 10000
```


After running the above query, we got the following set as a result, which we then exported as a CSV:

```
fullVisitorId, visitNumber, action_type, productSKU, time, hitNumber
0049931492016965831,1,1,GGOEGEVA022399,96360,6
0049931492016965831,1,2,GGOEGEVA022399,96361,7
0049931492016965831,1,1,GGOEGEVA022399,106182,8

```

Some data preprocessing was done in the SQl part, before the data was extracted. For example, whenever the action_type was 0 (unknown action) we filtered it out. Since we cannot really extract any information from that. In addition, the result was sorted in ascending order, so as to avoid sorting during the next steps.



## Install and Import Dependencies

In [ ]:
!pip install pyspark

from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import lead
from pyspark.sql.functions import when
from pyspark.sql.functions import sum

## Helper Functions

In [ ]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

# Useful functions to print RDDs and Dataframes.
def toCSVLineRDD(rdd):
    '''
    This function convert an RDD or a DataFrame into a CSV string
    '''
    a = rdd.map(lambda row: ",".join([str(elt) for elt in row]))\
           .reduce(lambda x, y: '\n'.join([x,y]))
    return a + '\n'

def toCSVLine(data):
    '''
    Convert an RDD or a DataFrame into a CSV string
    '''
    if isinstance(data, RDD):
        return toCSVLineRDD(data)
    elif isinstance(data, DataFrame):
        return toCSVLineRDD(data.rdd)
    return None

## Function To Calculate The Session Duration

This function calculates the time spent on product-detail pages. It does so by subtracting the time once the product was clicked on from time of the next page.

In [ ]:
def calculate_session_duration(filename):

    # Start a Spark session
    spark = init_spark()

    # Read data from file
    df = spark.read.csv(filename, header=True, inferSchema=True)

    # Create 'userId' by concatenating 'fullVisitorID' and 'visitNumber', separated by '-'
    df = df.withColumn("userId", concat(col("fullVisitorID"), lit("-"), col("visitNumber")))

    # Define window spec for ordering by time within each user session
    windowSpec = Window.partitionBy("userId").orderBy("time")

    # Calculate the difference in time for consecutive hits to get the pageview duration
    # First create a new column called next_hit_time, then initialize it with the the time of the next hit, from the next row, given it has the same userId
    # Then obtain the session duration from doing next_hit_time - time
    df = df.withColumn("next_hit_time", lead("time", 1).over(windowSpec))
    df = df.withColumn("pageview_duration", when(col("next_hit_time").isNull(), 0).otherwise(col("next_hit_time") - col("time")))

    # Filter for product detail views (where eCommerceAction.action_type == "2")
    prod_view_df = df.filter(df['action_type'] == '2')

    # Aggregate session durations by userId and productSKU, summing up durations to get total session duration
    aggregate_web_stats = prod_view_df.groupBy("userId", "productSKU").agg(sum("pageview_duration").alias("session_duration"))

    # Rename column to itemId
    aggregate_web_stats = aggregate_web_stats.withColumnRenamed("productSKU", "itemId")

    # Show the a sample of the result
    aggregate_web_stats.show(10)

    # write data to disk
    aggregate_web_stats.write.csv('../data/data_v1.csv', header=True, mode='overwrite')

## Ouput: Utility Matrix

```
userId,itemId,session_duration
1005829299685984449-1,GGOEYFKQ020699,58012
1005829299685984449-1,GGOEGGCX056299,787
```